modified from the unbond-generic example on hackage

if the import does not work you should run `stack build` from `pwd` directory.

In [1]:
{-# LANGUAGE DeriveDataTypeable
           , DeriveGeneric
           , MultiParamTypeClasses #-}

import Unbound.Generics.LocallyNameless
import GHC.Generics (Generic)
import Data.Typeable (Typeable)

In [2]:
-- | Names for expressions
type Nm = Name Exp

-- | Expressions
data Exp = Var Nm            -- ^ variables
         | Lam (Bind Nm Exp) -- ^ lambdas bind a variable within a body expression
         | App Exp Exp       -- ^ application
          deriving (Show, Generic, Typeable)

-- Automatically construct alpha equiv, free vars, and binding operations.
instance Alpha Exp

-- semi-automatically implement capture avoiding substitution
instance Subst Exp Exp where
  -- `isvar` identifies the variable case in your AST.
  isvar (Var x) = Just (SubstName x)
  isvar _       = Nothing

In [3]:
-- call-by-value evaluation
-- evaluation takes an expression and returns a value while using a source of fresh names
eval :: Exp -> FreshM Exp
eval (Var x)     = fail $ "unbound variable " ++ show x
eval e@(Lam {})  = return e
eval (App e1 e2) = do
  v1 <- eval e1
  v2 <- eval e2
  case v1 of
   (Lam bnd) -> do
     -- open the lambda by picking a fresh name for the bound variable x in body
     (x, body) <- unbind bnd
     eval (subst x v2 body)
   _ -> fail "application of non-lambda"

In [4]:
x = s2n "x"
y = s2n "y"
e = Lam $ bind x (Lam $ bind y (App (Var y) (Var x)))

In [5]:
e

Lam (<x> Lam (<y> App (Var 0@0) (Var 1@0)))

In [6]:
runFreshM $ eval (App (App e e) e)

Lam (<y> App (Var 0@0) (Lam (<x> Lam (<y> App (Var 0@0) (Var 1@0)))))